In [ ]:
import pandas as pd

# Load the CSV file
data1 = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')

texts = data1['full_text'].tolist()
text = ' '.join(texts)
len(text)

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lr_scheduler

# hyperparameters
batch_size = 32 #how many independent sequences will we process in parallel?
block_size = 512 # what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 5000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 10
n_embd = 768
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    # generate a batch of data for inputs and targets
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size * 2, (batch_size,))
    
    # Encoder input
    encoder_inputs = torch.stack([data[i:i + block_size] for i in ix])
    # Decoder input
    decoder_inputs = torch.stack([data[i + block_size:i + block_size * 2] for i in ix])
    # Target sequence for the decoder
    targets = torch.stack([data[i + block_size + 1:i + block_size * 2 + 1] for i in ix])

    encoder_inputs, decoder_inputs, targets = encoder_inputs.to(device), decoder_inputs.to(device), targets.to(device)
    return encoder_inputs, decoder_inputs, targets

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            encoder_inputs, decoder_inputs, targets = get_batch(split)
            logits = model(encoder_inputs, decoder_inputs)
            loss = model.calculate_loss(logits, targets)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size, n_embd, block_size, dropout):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, k, q, v, mask=False):
        B, T, C = k.shape
        k = self.key(k)   # (B, T, hs)
        q = self.query(q) # (B, T, hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        if mask != False:
            wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(v)  # (B, T, hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size, n_embd, block_size, dropout):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=False):
        out = torch.cat([h(x, x, x, mask) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class EncoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, n_embd, block_size, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class DecoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, block_size, dropout):
        super().__init__()
        head_size = n_embd // n_head
        self.msa = MultiHeadAttention(n_head, head_size, n_embd, block_size, dropout)
        self.ca = MultiHeadAttention(n_head, head_size, n_embd, block_size, dropout) # CROSS ATTENTION HEADS
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ln3 = nn.LayerNorm(n_embd)
        

    def forward(self, x, enc_output):
        x = x + self.msa(self.ln1(x), mask=True)
        x = x + self.ca(self.ln2(enc_output), mask=False) #CROSS ATTENTION
        x = x + self.ffwd(self.ln3(x))
        return x

class GPTTransformer(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, block_size, n_layer, dropout):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Embedding(block_size, n_embd)
        self.encoder = nn.ModuleList([EncoderBlock(n_embd, n_head, block_size, dropout) for _ in range(n_layer)])
        self.decoder = nn.ModuleList([DecoderBlock(n_embd, n_head, block_size, dropout) for _ in range(n_layer)])
        self.proj = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
        
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, src, tgt):
        B, T = src.size()
        device = src.device
        pos_id = torch.arange(T, dtype=torch.long, device=device)
        pos_id = pos_id.unsqueeze(0).expand(B, -1)
        pos_emb = self.pos_emb(pos_id)

        src_emb = self.token_emb(src) + pos_emb
        tgt_emb = self.token_emb(tgt) + pos_emb

        enc_output = src_emb
        for enc_block in self.encoder:
            enc_output = enc_block(enc_output)

        dec_output = tgt_emb
        for dec_block in self.decoder:
            dec_output = dec_block(dec_output, enc_output)

        logits = self.proj(dec_output)
        return logits

    def calculate_loss(self, logits, targets):
        # Flatten logits and targets to shape (batch_size * sequence_length, vocab_size)
        logits_flat = logits.view(-1, logits.size(-1))
        targets_flat = targets.view(-1)

        # Calculate cross-entropy loss
        loss = F.cross_entropy(logits_flat, targets_flat, ignore_index=0)  # Ignore padding index
        return loss
    
    def generate(self, src, max_new_tokens):
        batch_size = src.size(0)
        # Initialize the decoder input with start tokens
        decoder_inputs = torch.full((batch_size, 1), start_token_idx, dtype=torch.long, device=device)

        for _ in range(max_new_tokens):
            # Concatenate source sequences with decoder inputs
            inputs = torch.cat((src, decoder_inputs), dim=1)

            # Get predictions from the model
            logits = self(inputs)  # Assuming self(inputs) returns logits directly without calculating loss

            # Focus only on the last time step
            logits = logits[:, -1, :]  # (B, C)

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # Append sampled index to the running sequence
            decoder_inputs = torch.cat((decoder_inputs, idx_next), dim=1)  # (B, T+1)

        return decoder_inputs

In [ ]:
model = GPTTransformer(vocab_size=vocab_size, n_embd=n_embd, n_head=n_head, block_size=block_size, n_layer=n_layer, dropout=dropout)
m = model.to(device)

# PRINTING THE NUMBER OF PARAMETERS IN THE MODEL
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # PRINTING LOSS EVERY SPECIFIC INTERVAL
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # SAMPLING A BATCH OF DATA
    encoder_inputs, decoder_inputs, targets = get_batch('train')

    # EVALUATING THE LOSS
    logits = model(encoder_inputs, decoder_inputs)
    loss = model.calculate_loss(logits, targets)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
# GENERATE FROM THE MODEL
encoder_inputs = torch.zeros((1, 1), dtype=torch.long, device=device)
decoder_outputs = m.generate(encoder_inputs, max_new_tokens=500)
generated_text = decode(decoder_outputs[0].tolist())
print(generated_text)

In [ ]:
# SAVE THE MODEL
torch.save(model.state_dict(), '/kaggle/working/gpt100M_50k.pth')